In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Step 1: Load Time Series Dataset
# For demonstration, let's use some random time series data
# Replace this with your actual dataset
# Example:
# df = pd.read_csv('your_dataset.csv')

# Generating random data for demonstration
np.random.seed(42)
date_range = pd.date_range(start='1/1/2010', end='12/31/2019', freq='D')
time_series_data = np.random.normal(loc=100, scale=20, size=len(date_range))
df = pd.DataFrame({'Date': date_range, 'Value': time_series_data})

# Step 2: Preprocess the Data
# Convert date to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Sort dataframe by date
df.sort_values('Date', inplace=True)

# Normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['Value'].values.reshape(-1, 1))

# Function to create dataset with look back
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), 0])
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 60  # Number of previous time steps to use as input variables
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]
train_X, train_Y = create_dataset(train_data, look_back)
test_X, test_Y = create_dataset(test_data, look_back)

# Reshape input to be [samples, time steps, features]
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))
test_X = np.reshape(test_X, (test_X.shape[0], test_X.shape[1], 1))

# Step 3: Build the LSTM Model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(train_X.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Step 4: Train the Model
model.fit(train_X, train_Y, epochs=100, batch_size=32)

# Step 5: Evaluate the Model
# Predictions
train_predictions = model.predict(train_X)
test_predictions = model.predict(test_X)

# Inverse transform the predictions and actual values
train_predictions = scaler.inverse_transform(train_predictions)
train_actual = scaler.inverse_transform([train_Y])
test_predictions = scaler.inverse_transform(test_predictions)
test_actual = scaler.inverse_transform([test_Y])

# Calculate MAE and RMSE
train_mae = mean_absolute_error(train_actual[0], train_predictions[:, 0])
train_rmse = np.sqrt(mean_squared_error(train_actual[0], train_predictions[:, 0]))
test_mae = mean_absolute_error(test_actual[0], test_predictions[:, 0])
test_rmse = np.sqrt(mean_squared_error(test_actual[0], test_predictions[:, 0]))

print("Train MAE:", train_mae)
print("Train RMSE:", train_rmse)
print("Test MAE:", test_mae)
print("Test RMSE:", test_rmse)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

# Step 1: Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Step 2: Build a simple convolutional neural network (CNN)
def create_model(learning_rate=0.001, batch_size=64, epochs=10):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Step 3: Use grid search to optimize hyperparameters
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [32, 64, 128],
    'epochs': [5, 10, 15]
}

best_accuracy = 0
best_params = {}

for learning_rate in param_grid['learning_rate']:
    for batch_size in param_grid['batch_size']:
        for epochs in param_grid['epochs']:
            print(f"Training model with learning_rate={learning_rate}, batch_size={batch_size}, epochs={epochs}")
            model = create_model(learning_rate=learning_rate, batch_size=batch_size, epochs=epochs)
            history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=2)
            test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
            print("Test Accuracy:", test_accuracy)
            if test_accuracy > best_accuracy:
                best_accuracy = test_accuracy
                best_params = {'learning_rate': learning_rate, 'batch_size': batch_size, 'epochs': epochs}

# Print best parameters and accuracy
print("Best Parameters:", best_params)
print("Best Test Accuracy:", best_accuracy)

# Step 4: Use Callback functions
model = create_model(**best_params)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
history = model.fit(X_train, y_train, batch_size=best_params['batch_size'], epochs=best_params['epochs'], validation_data=(X_test, y_test), callbacks=[reduce_lr], verbose=2)

# Step 5: Visualize results using history object
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()


Training model with learning_rate=0.001, batch_size=32, epochs=5
Epoch 1/5
1875/1875 - 26s - 14ms/step - accuracy: 0.9613 - loss: 0.1271 - val_accuracy: 0.9844 - val_loss: 0.0458
Epoch 2/5
1875/1875 - 23s - 12ms/step - accuracy: 0.9869 - loss: 0.0414 - val_accuracy: 0.9893 - val_loss: 0.0298
Epoch 3/5
1875/1875 - 23s - 12ms/step - accuracy: 0.9907 - loss: 0.0290 - val_accuracy: 0.9914 - val_loss: 0.0250
Epoch 4/5
